In [2]:
import glob
import torch
import torch.nn as nn
from torchvision import models
from PIL import Image

import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import torch.utils.data as data
from torchvision import models, transforms
import torch.optim as optim
from torchvision import models
from tqdm import tqdm

import numpy as np
import random

In [3]:
#訓練画像、評価画像のパスは以下
train_img_path = "/kaggle/input/ranzcr-clip-catheter-line-classification/train/*.jpg"
test_img_path = "/kaggle/input/ranzcr-clip-catheter-line-classification/test/*.jpg"

# 訓練画像アノテーションのパス
train_annot_path = "/kaggle/input/ranzcr-clip-catheter-line-classification/train.csv"

In [4]:
## 画像フォルダから画像パス一括読み込み

train_imgs = glob.glob(train_img_path)
test_imgs = glob.glob(test_img_path)

In [5]:
## パス確認
for train,test in zip(train_imgs,test_imgs):
    #train_img = Image.open(train)
    #test_img = Image.open(test)
    
    print("train path:{}¥ntest path:{}".format(train,test))

train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.10370758874574386468962321364924311754.jpg¥ntest path:/kaggle/input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.24641136930096467169760392302420182106.jpg
train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.84444065890720277399079723469279075346.jpg¥ntest path:/kaggle/input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.12690617441924311870163706574398714392.jpg
train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.57971305277490414138568914293456437940.jpg¥ntest path:/kaggle/input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.12475334287210977140172691428186327103.jpg
train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.84952542212696927819223009790147228632.jpg¥ntest path:/kagg

train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.94160295086836719424718676281303573148.jpg¥ntest path:/kaggle/input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.11552295077327172274621483823826389557.jpg
train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.11556153841367014710401380338884380143.jpg¥ntest path:/kaggle/input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.13782042952975122601267912402559716796.jpg
train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.43604003287723258135502309399925353789.jpg¥ntest path:/kaggle/input/ranzcr-clip-catheter-line-classification/test/1.2.826.0.1.3680043.8.498.14471383470138765491594027258133884276.jpg
train path:/kaggle/input/ranzcr-clip-catheter-line-classification/train/1.2.826.0.1.3680043.8.498.92202717822452996522572351077914710916.jpg¥ntest path:/kagg

In [6]:
print(len(train_imgs))
print(len(test_imgs))

30083
3582


In [7]:
train_annot = pd.read_csv(train_annot_path)



train_ett_annot = train_annot.iloc[:,1:4]

print(train_ett_annot)

       ETT - Abnormal  ETT - Borderline  ETT - Normal
0                   0                 0             0
1                   0                 0             1
2                   0                 0             0
3                   0                 0             0
4                   0                 0             0
...               ...               ...           ...
30078               0                 0             1
30079               0                 0             0
30080               0                 0             1
30081               0                 0             0
30082               0                 0             1

[30083 rows x 3 columns]


# 前処理クラス ImageTransform
* 画像データをTensor型に変換
* データ拡張

In [8]:
class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)

In [9]:
def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'val'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = "/kaggle/input/ranzcr-clip-catheter-line-classification"
    if phase == "train":
        target_path = osp.join(rootpath,phase+'/*.jpg')
    else:
        target_path = osp.join(rootpath,phase+'/*.jpg')
    
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)
    
    num_train = len(path_list)*0.8
    
    train_path_list = []
    val_path_list = []
    
    if phase == "train":
        return path_list[:int(num_train)], path_list[int(num_train):]
    else:
        return path_list


In [19]:
class RanzcrDataset(data.Dataset):
    """
    Datasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    """

    def __init__(self, file_list, annotation_path,transform=None, phase='train'):
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定
        self.annotation_path = annotation_path ## アノテーションデータ

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]
        img = img.convert("RGB")

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])
        
        annot_data = pd.read_csv(self.annotation_path)
        
        img_name = img_path.split("/")[-1].strip(".jpg")
        #print(img_name)
        
        annot = annot_data[annot_data["StudyInstanceUID"]==img_name]
        #print("img_name;{}¥tannot:{}".format(img_name, annot.iloc[:,1:4]))
        label = annot.iloc[:,1:4].values
        label = label[0].reshape(-1)
        #print(np.argmax(label))
        #lable = torch.from_numpy(label.astype(np.float32)).clone()
        #print(label.shape)
        #print(label)
        #label = torch.from_numpy(label.astype(np.uint8)).clone()
        #label = torch.unsqueeze(label,0)
        
        return img_transformed, label
        #return img_transformed, np.argmax(label)

In [20]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [21]:
# 訓練、テストデータのアノテーションパス
train_annot_path = "/kaggle/input/ranzcr-clip-catheter-line-classification/train.csv"

# 訓練, テストの画像へのファイルパスのリストを作成する
train_list,val_list = make_datapath_list(phase="train")
#val_list = make_datapath_list(phase="val")

print(len(train_list), len(val_list))

# Datasetを作成する
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = RanzcrDataset(
    file_list=train_list, annotation_path=train_annot_path,
    transform=ImageTransform(size, mean, std), phase='train')

val_dataset = RanzcrDataset(
    file_list=val_list, annotation_path=train_annot_path,
    transform=ImageTransform(size, mean, std), phase='val')

# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)


# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

/kaggle/input/ranzcr-clip-catheter-line-classification/train/*.jpg
24066 6017


In [22]:
# 学習済みのVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの2つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=3)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


In [23]:
# 損失関数の設定
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()

In [24]:
# 最適化手法の設定
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [27]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in dataloaders_dict[phase]:

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)
                
            
                
                #print(labels)
                
                  # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    #epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            #epoch_acc = epoch_corrects.double(
            #) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            print('{} Loss: {:.4f}'.format(
                phase, epoch_loss))



In [28]:
# 学習・検証を実行する
num_epochs=20
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
Epoch 1/20
-------------


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:
# PyTorchのネットワークパラメータの保存
save_path = './ranzcr_vgg16_weights_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [ ]:
# PyTorchのネットワークパラメータのロード
load_path = './ranzcr_vgg16_weights_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

# GPU上で保存された重みをCPU上でロードする場合
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights)